# Create Hbond network data

In [1]:
import mdtraj as md
import pandas as pd
import numpy as np
import MDAnalysis as mda
import MDAnalysis.analysis.hbonds as hbonds
import matplotlib
import matplotlib.pyplot as plt
import ast


In [2]:
traj_notes = ['5NJY_pH70_md1','5NJY_pH70_md2','5NJY_pH70_md3','5NJY_F238L_pH70_md1',
                 '5NJY_F238L_pH70_md2','5NJY_F238L_pH70_md3','5NJY_I233T_pH70_md4',
                  '5NJY_I233T_pH70_md2','5NJY_I233T_pH70_md3','5NJY_F238LI233T_pH70_md1',
                  '5NJY_F238LI233T_pH70_md2','5NJY_F238LI233T_pH70_md3','5NJY_pH46_md1','5NJY_pH46_md2','5NJY_pH46_md3','5NJY_F238L_pH46_md1',
                 '5NJY_F238L_pH46_md2','5NJY_F238L_pH46_md3','5NJY_I233T_pH46_md1',
                  '5NJY_I233T_pH46_md2','5NJY_I233T_pH46_md3','5NJY_F238LI233T_pH46_md1',
                  '5NJY_F238LI233T_pH46_md2','5NJY_F238LI233T_pH46_md3','4HFI_pH46_md1','4HFI_pH46_md2','4HFI_pH46_md3','4HFI_F238L_pH46_md1',
                 '4HFI_F238L_pH46_md2','4HFI_F238L_pH46_md3','4HFI_I233T_pH46_md1',
                  '4HFI_I233T_pH46_md2','4HFI_I233T_pH46_md3','4HFI_F238LI233T_pH46_md1',
                  '4HFI_F238LI233T_pH46_md2','4HFI_F238LI233T_pH46_md3','4NPQ_pH70_md5','4NPQ_pH70_md6','4NPQ_pH70_md7','4NPQ_F238L_pH70_md3',
                 '4NPQ_F238L_pH70_md4','4NPQ_F238L_pH70_md5','4NPQ_I233T_pH70_md3',
                  '4NPQ_I233T_pH70_md4','4NPQ_I233T_pH70_md5','4NPQ_F238LI233T_pH70_md3',
                  '4NPQ_F238LI233T_pH70_md4','4NPQ_F238LI233T_pH70_md5']

In [3]:
def create_Hbond_network_analysis_data(traj_note, location = '/media/scottzhuang/data/MD/', skip=10):
    top_location = traj_note + '/' + traj_note + ".system.pdb"
    traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".system.xtc"
    traj = md.load(location + traj_location,top = location + top_location)
    topology = traj.topology
    table, bonds = topology.to_dataframe()
    n_frames = traj.n_frames
    zero_data = np.zeros(shape=([118*5,118*5]))
    hbond_network_dataframe = pd.DataFrame()
    for time in range(0,n_frames*10000,10000):  
        hbond_network = pd.DataFrame(zero_data,columns=np.asarray([np.arange(193,311),np.arange(193,311) + 311,np.arange(193,311) + 311 * 2,np.arange(193,311) + 311 * 3,np.arange(193,311) + 311 * 4]).flatten())
        hbond_network['traj_time'] = time/10000
        hbond_network['residue'] = np.asarray([np.arange(193,311),np.arange(193,311) + 311,np.arange(193,311) + 311 * 2,np.arange(193,311) + 311 * 3,np.arange(193,311) + 311 * 4]).flatten()
        hbond_network_dataframe = hbond_network_dataframe.append(hbond_network,ignore_index=True)
    for time in range(0,n_frames*10000,10000):  
        for resid in range(198,315):
            hbond_dataframe = pd.read_csv(location + traj_note + '/' + str(resid) + '_hbond_list.csv')
            hbond_dataframe['counterpart'] = hbond_dataframe['counterpart'].apply(lambda x: ast.literal_eval(x))
            water_bridge_dataframe = pd.read_csv(location + traj_note + '/' + str(resid) + '_water_bridge_list.csv')
            water_bridge_dataframe['counterpart'] = water_bridge_dataframe['counterpart'].apply(lambda x: ast.literal_eval(x))

            for residue_id in set(hbond_dataframe['resid']):
                counterpart_residue = hbond_dataframe[(hbond_dataframe['resid'] == residue_id) & (hbond_dataframe['traj_time'] == time)]['counterpart']
                for counterpart_residue_list in counterpart_residue:
                    for counterpart_residue_id in counterpart_residue_list:
                        if (table.loc[counterpart_residue_id].resSeq >= 198) & (table.loc[counterpart_residue_id].resSeq <= 315) & (table.loc[residue_id].resSeq >= 198) & (table.loc[residue_id].resSeq <= 315):
                            residue1 = table.loc[residue_id].resSeq + (table.loc[residue_id].chainID) * 311 - 5
                            residue2 = table.loc[counterpart_residue_id].resSeq + (table.loc[counterpart_residue_id].chainID) * 311 - 5

                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time/10000) & (hbond_network_dataframe.residue == residue1),residue2] = 1
            
            for residue_id in set(water_bridge_dataframe['resid']):
                counterpart_residue = water_bridge_dataframe[(water_bridge_dataframe['resid'] == residue_id) & (water_bridge_dataframe['traj_time'] == time)]['counterpart'].values
                for counterpart_residue_list in counterpart_residue:
                    for counterpart_residue_id in counterpart_residue_list:
                        if (table.loc[counterpart_residue_id].resSeq >= 198) & (table.loc[counterpart_residue_id].resSeq <= 315) & (table.loc[residue_id].resSeq >= 198) & (table.loc[residue_id].resSeq <= 315):
                            residue1 = table.loc[residue_id].resSeq + (table.loc[residue_id].chainID) * 311 - 5
                            residue2 = table.loc[counterpart_residue_id].resSeq + (table.loc[counterpart_residue_id].chainID) * 311 - 5
                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time /10000) & (hbond_network_dataframe.residue == residue1),residue2] = 1
    hbond_network_dataframe.to_csv(location + traj_note + '/hbond_network.csv')
    print('finishing!')
   #         print('At time ' + str(time) + ', in total ' + str(n_hbond) + ' hydrogen bonds between residues, resid '
   #               + str(resid) + ' forms ' + str(n_intra_hbond) + ' intra-subunit H-bond and ' + str(n_inter_hbond) + ' inter-subunit H-bond.')

In [29]:
def create_Hbond_network_analysis_data(traj_note, location = '/media/scottzhuang/data/MD/', skip=10):
    top_location = traj_note + '/' + traj_note + ".system.pdb"
    traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".system.xtc"
    traj = md.load(location + traj_location,top = location + top_location)
    topology = traj.topology
    table, bonds = topology.to_dataframe()
    n_frames = traj.n_frames
    zero_data = np.zeros(shape=([118*5,118*5]))
    hbond_network_dataframe = pd.DataFrame()
    for time in range(0,n_frames*10000,10000):  
        hbond_network = pd.DataFrame(zero_data,columns=np.asarray([np.arange(193,311),np.arange(193,311) + 311,np.arange(193,311) + 311 * 2,np.arange(193,311) + 311 * 3,np.arange(193,311) + 311 * 4]).flatten())
        hbond_network['traj_time'] = time/10000
        hbond_network['residue'] = np.asarray([np.arange(193,311),np.arange(193,311) + 311,np.arange(193,311) + 311 * 2,np.arange(193,311) + 311 * 3,np.arange(193,311) + 311 * 4]).flatten()
        hbond_network_dataframe = hbond_network_dataframe.append(hbond_network,ignore_index=True)
    for time in range(0,n_frames*10000,10000):  
        for resid in range(198,315):
            hbond_dataframe = pd.read_csv(location + traj_note + '/hbond_analysis/' + str(resid) + '_hbond_list.csv')
            hbond_dataframe['counterpart'] = hbond_dataframe['counterpart'].apply(lambda x: ast.literal_eval(x))
            water_bridge_dataframe = pd.read_csv(location + traj_note + '/hbond_analysis/' + str(resid) + '_water_bridge_list.csv')
            water_bridge_dataframe['counterpart'] = water_bridge_dataframe['counterpart'].apply(lambda x: ast.literal_eval(x))

            for residue_id in set(hbond_dataframe['resid']):
                counterpart_residue = hbond_dataframe[(hbond_dataframe['resid'] == residue_id) & (hbond_dataframe['traj_time'] == time)]['counterpart']
                for counterpart_residue_list in counterpart_residue:
                    for counterpart_residue_id in counterpart_residue_list:
                        if (table.loc[counterpart_residue_id].resSeq >= 198) & (table.loc[counterpart_residue_id].resSeq <= 315) & (table.loc[residue_id].resSeq >= 198) & (table.loc[residue_id].resSeq <= 315):
                            residue1 = table.loc[residue_id].resSeq + (table.loc[residue_id].chainID) * 311 - 5
                            residue2 = table.loc[counterpart_residue_id].resSeq + (table.loc[counterpart_residue_id].chainID) * 311 - 5

                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time/10000) & (hbond_network_dataframe.residue == residue1),residue2] = 1
                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time/10000) & (hbond_network_dataframe.residue == residue2),residue1] = 1

            for residue_id in set(water_bridge_dataframe['resid']):
                counterpart_residue = water_bridge_dataframe[(water_bridge_dataframe['resid'] == residue_id) & (water_bridge_dataframe['traj_time'] == time)]['counterpart'].values
                for counterpart_residue_list in counterpart_residue:
                    for counterpart_residue_id in counterpart_residue_list:
                        if (table.loc[counterpart_residue_id].resSeq >= 198) & (table.loc[counterpart_residue_id].resSeq <= 315) & (table.loc[residue_id].resSeq >= 198) & (table.loc[residue_id].resSeq <= 315):
                            residue1 = table.loc[residue_id].resSeq + (table.loc[residue_id].chainID) * 311 - 5
                            residue2 = table.loc[counterpart_residue_id].resSeq + (table.loc[counterpart_residue_id].chainID) * 311 - 5
                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time /10000) & (hbond_network_dataframe.residue == residue1),residue2] = 1
                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time /10000) & (hbond_network_dataframe.residue == residue2),residue1] = 1

    hbond_network_dataframe.to_csv(location + traj_note + '/hbond_analysis/hbond_network_2.csv')
    print('finishing!')
   #         print('At time ' + str(time) + ', in total ' + str(n_hbond) + ' hydrogen bonds between residues, resid '
   #               + str(resid) + ' forms ' + str(n_intra_hbond) + ' intra-subunit H-bond and ' + str(n_inter_hbond) + ' inter-subunit H-bond.')

In [12]:
def create_Hbond_network_analysis_data_robust(traj_note,i, location = '/media/scottzhuang/data/MD/', skip=10):
    top_location = traj_note + '/' + traj_note + ".system.pdb"
    traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".system.xtc"
    traj = md.load(location + traj_location,top = location + top_location)
    topology = traj.topology
    table, bonds = topology.to_dataframe()
    n_frames = traj.n_frames
    zero_data = np.zeros(shape=([118*5,118*5]))
    hbond_network_dataframe = pd.DataFrame()
    for time in range(int(n_frames*i/3)*10000,int(n_frames*(i+1)/3)*10000,10000):  
        hbond_network = pd.DataFrame(zero_data,columns=np.asarray([np.arange(193,311),np.arange(193,311) + 311,np.arange(193,311) + 311 * 2,np.arange(193,311) + 311 * 3,np.arange(193,311) + 311 * 4]).flatten())
        hbond_network['traj_time'] = time/10000
        hbond_network['residue'] = np.asarray([np.arange(193,311),np.arange(193,311) + 311,np.arange(193,311) + 311 * 2,np.arange(193,311) + 311 * 3,np.arange(193,311) + 311 * 4]).flatten()
        hbond_network_dataframe = hbond_network_dataframe.append(hbond_network,ignore_index=True)
    for time in range(int(n_frames*i/3)*10000,int(n_frames*(i+1)/3)*10000,10000):  
        for resid in range(198,315):
            hbond_dataframe = pd.read_csv(location + traj_note + '/hbond_analysis/' + str(resid) + '_hbond_list.csv')
            hbond_dataframe['counterpart'] = hbond_dataframe['counterpart'].apply(lambda x: ast.literal_eval(x))
            water_bridge_dataframe = pd.read_csv(location + traj_note + '/hbond_analysis/' + str(resid) + '_water_bridge_list.csv')
            water_bridge_dataframe['counterpart'] = water_bridge_dataframe['counterpart'].apply(lambda x: ast.literal_eval(x))

            for residue_id in set(hbond_dataframe['resid']):
                counterpart_residue = hbond_dataframe[(hbond_dataframe['resid'] == residue_id) & (hbond_dataframe['traj_time'] == time)]['counterpart']
                for counterpart_residue_list in counterpart_residue:
                    for counterpart_residue_id in counterpart_residue_list:
                        if (table.loc[counterpart_residue_id].resSeq >= 198) & (table.loc[counterpart_residue_id].resSeq <= 315) & (table.loc[residue_id].resSeq >= 198) & (table.loc[residue_id].resSeq <= 315):
                            residue1 = table.loc[residue_id].resSeq + (table.loc[residue_id].chainID) * 311 - 5
                            residue2 = table.loc[counterpart_residue_id].resSeq + (table.loc[counterpart_residue_id].chainID) * 311 - 5

                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time/10000) & (hbond_network_dataframe.residue == residue1),residue2] = 1
                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time/10000) & (hbond_network_dataframe.residue == residue2),residue1] = 1

            for residue_id in set(water_bridge_dataframe['resid']):
                counterpart_residue = water_bridge_dataframe[(water_bridge_dataframe['resid'] == residue_id) & (water_bridge_dataframe['traj_time'] == time)]['counterpart'].values
                for counterpart_residue_list in counterpart_residue:
                    for counterpart_residue_id in counterpart_residue_list:
                        if (table.loc[counterpart_residue_id].resSeq >= 198) & (table.loc[counterpart_residue_id].resSeq <= 315) & (table.loc[residue_id].resSeq >= 198) & (table.loc[residue_id].resSeq <= 315):
                            residue1 = table.loc[residue_id].resSeq + (table.loc[residue_id].chainID) * 311 - 5
                            residue2 = table.loc[counterpart_residue_id].resSeq + (table.loc[counterpart_residue_id].chainID) * 311 - 5
                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time /10000) & (hbond_network_dataframe.residue == residue1),residue2] = 1
                            hbond_network_dataframe.loc[(hbond_network_dataframe.traj_time == time /10000) & (hbond_network_dataframe.residue == residue2),residue1] = 1

    hbond_network_dataframe.to_csv(location + traj_note + '/hbond_analysis/hbond_network_robust_' + str(i) + '.csv')
    print('finishing!')
   #         print('At time ' + str(time) + ', in total ' + str(n_hbond) + ' hydrogen bonds between residues, resid '
   #               + str(resid) + ' forms ' + str(n_intra_hbond) + ' intra-subunit H-bond and ' + str(n_inter_hbond) + ' inter-subunit H-bond.')

In [3]:
def create_hbond_contact_map(traj_note, location = '/media/scottzhuang/data/MD/', skip=10):
    top_location = traj_note + '/' + traj_note + ".ca.pdb"
    traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
    traj = md.load(location + traj_location,top = location + top_location)
    n_frames = traj.n_frames
    hbond_contact_map = pd.DataFrame(columns=['residue1','residue2','interaction'])
    hbond_network_dataframe = pd.read_csv(location + traj_note + '/hbond_analysis/hbond_network_2.csv')
    for column in hbond_network_dataframe.columns:
        if (column !=  'traj_time') & (column != 'residue') & (column != 'Unnamed: 0'):
            residue1s = hbond_network_dataframe.loc[(hbond_network_dataframe[column] != 0)]['residue'].values
            residue2 = int(column)
            for residue1 in set(residue1s):
                if hbond_contact_map[(hbond_contact_map.residue1 == residue1) & (hbond_contact_map.residue2 == residue2)].empty:
                    interaction_sum = hbond_network_dataframe[(hbond_network_dataframe[column] != 0) & (hbond_network_dataframe.residue == residue1)].shape[0]
                    hbond_contact_map = pd.concat([hbond_contact_map,pd.DataFrame([[residue1,residue2,interaction_sum / n_frames]],columns=['residue1','residue2','interaction'])],ignore_index=True)
                else:
                    interaction_sum = hbond_network_dataframe[(hbond_network_dataframe[column] != 0) & (hbond_network_dataframe.residue == residue1)].shape[0]
                    hbond_contact_map.loc[(hbond_contact_map.residue1 == residue1) & (hbond_contact_map.residue2 == residue2),'interaction'] += interaction_sum / n_frames
    #print(hbond_contact_map)
    hbond_contact_map.to_csv(location + traj_note + '/hbond_analysis/hbond_contact_map_2.csv')
    print('finishing! ' + traj_note)

In [19]:
def create_hbond_contact_map_robust(traj_note,i, location = '/media/scottzhuang/data/MD/', skip=10):
    top_location = traj_note + '/' + traj_note + ".ca.pdb"
    traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
    traj = md.load(location + traj_location,top = location + top_location)
    n_frames = traj.n_frames
    hbond_contact_map = pd.DataFrame(columns=['residue1','residue2','interaction'])
    hbond_network_dataframe = pd.read_csv(location + traj_note + '/hbond_analysis/hbond_network_2.csv')
    hbond_network_dataframe = hbond_network_dataframe[(hbond_network_dataframe.traj_time >= n_frames * i / 3 ) & (hbond_network_dataframe.traj_time <= n_frames *(i + 1) / 3 )]
    for column in hbond_network_dataframe.columns:
        if (column !=  'traj_time') & (column != 'residue') & (column != 'Unnamed: 0'):
            residue1s = hbond_network_dataframe.loc[(hbond_network_dataframe[column] != 0)]['residue'].values
            residue2 = int(column)
            for residue1 in set(residue1s):
                if hbond_contact_map[(hbond_contact_map.residue1 == residue1) & (hbond_contact_map.residue2 == residue2)].empty:
                    interaction_sum = hbond_network_dataframe[(hbond_network_dataframe[column] != 0) & (hbond_network_dataframe.residue == residue1)].shape[0]
                    hbond_contact_map = pd.concat([hbond_contact_map,pd.DataFrame([[residue1,residue2,interaction_sum * 3 / n_frames]],columns=['residue1','residue2','interaction'])],ignore_index=True)
                else:
                    interaction_sum = hbond_network_dataframe[(hbond_network_dataframe[column] != 0) & (hbond_network_dataframe.residue == residue1)].shape[0]
                    hbond_contact_map.loc[(hbond_contact_map.residue1 == residue1) & (hbond_contact_map.residue2 == residue2),'interaction'] += interaction_sum * 3 / n_frames
    #print(hbond_contact_map)
    hbond_contact_map.to_csv(location + traj_note + '/hbond_analysis/hbond_contact_map_robust_' + str(i) + '.csv')
    print('finishing! ' + traj_note)

In [20]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(create_hbond_contact_map_robust)(traj_note,i) for (traj_note,i) in itertools.product(traj_notes,[0,1,2]))

finishing! 5NJY_F238L_pH70_md1
finishing! 5NJY_F238L_pH70_md1
finishing! 5NJY_F238L_pH70_md2
finishing! 5NJY_F238L_pH70_md1
finishing! 5NJY_pH70_md2
finishing! 5NJY_F238L_pH70_md2
finishing! 5NJY_F238L_pH70_md2
finishing! 5NJY_pH70_md2
finishing! 5NJY_F238L_pH70_md3
finishing! 5NJY_pH70_md1
finishing! 5NJY_pH70_md3
finishing! 5NJY_pH70_md3
finishing! 5NJY_pH70_md3
finishing! 5NJY_pH70_md2
finishing! 5NJY_pH70_md1
finishing! 5NJY_pH70_md1
finishing! 5NJY_F238L_pH70_md3
finishing! 5NJY_F238L_pH70_md3
finishing! 5NJY_I233T_pH70_md4
finishing! 5NJY_I233T_pH70_md4
finishing! 5NJY_I233T_pH70_md4
finishing! 5NJY_I233T_pH70_md2
finishing! 5NJY_I233T_pH70_md2
finishing! 5NJY_I233T_pH70_md2
finishing! 5NJY_I233T_pH70_md3
finishing! 5NJY_I233T_pH70_md3
finishing! 5NJY_I233T_pH70_md3
finishing! 5NJY_F238LI233T_pH70_md1
finishing! 5NJY_F238LI233T_pH70_md1
finishing! 5NJY_F238LI233T_pH70_md2
finishing! 5NJY_F238LI233T_pH70_md1
finishing! 5NJY_F238LI233T_pH70_md2
finishing! 5NJY_F238LI233T_pH70_md2
f

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [47]:
def create_hbond_network_max_rep(traj_note, location = '/media/scottzhuang/data/MD/', skip=10):
    contact_map = pd.read_csv(location + traj_note + '/hbond_analysis/hbond_contact_map_complete_2.csv')
    
    Hbond_network = nx.Graph()
    #Hbond_network.add_nodes_from(contact_map.residue1.values.astype(int))
    #Hbond_network.add_nodes_from(contact_map.residue2.values.astype(int))
    Hbond_network.add_nodes_from(np.arange(0,1555))
    pos_spring = nx.spring_layout(Hbond_network)
    pos_circular = nx.circular_layout(Hbond_network)
    #print(Hbond_network.number_of_nodes())
    for index,row in contact_map.iterrows():
        if (row['hbond'] >= 0.5)  & (row['residue1'] in pos_helix.keys()) & (row['residue2'] in pos_helix.keys()) :
            Hbond_network.add_edge(row['residue1'],row['residue2'],weight= row['hbond'])


    #print(Hbond_network.number_of_edges())
    Hbond_network = max(nx.connected_component_subgraphs(Hbond_network), key=len)
    resid233 = np.arange(0,5) * 311 + 233 - 5
    resid238 = np.arange(0,5) * 311 + 238 - 5 
    
    plt.subplot(121,aspect='equal')    
    weights = [Hbond_network[u][v]['weight'] * 3 for u,v in Hbond_network.edges]
    options = {'node_color': 'cyan','node_size': 10,'width': weights,'edge_color':weights,'font_size':1,'edge_cmap':plt.cm.Blues,'vmax':1,'vmin':0}
#    options = {'node_color': 'cyan','node_size': 10,'width': 3,'edge_color':'r','font_size':1}
    nx.draw_networkx_edges(Hbond_network,pos_helix, with_labels=False,**options)
    for residue in resid233:
        if residue in list(Hbond_network.nodes):
            nx.draw_networkx_nodes(Hbond_network, pos_helix, node_size=20, nodelist=[residue],node_color='red')
    for residue in resid238:
        if residue in list(Hbond_network.nodes):
            nx.draw_networkx_nodes(Hbond_network, pos_helix, node_size=20, nodelist=[residue],node_color='green')
    
#    for edge in list(Hbond_network.edges):
#        if (edge[0] in helix_1_ind[1]) & (edge[1] in helix_2_ind[4]):
#            print(edge)
    plt.scatter(h1.T[0],h1.T[1],color='black',alpha=0.5)
    plt.scatter(h2.T[0],h2.T[1],color='black',alpha=0.5)
    plt.scatter(h3.T[0],h3.T[1],color='black',alpha=0.5)
    plt.scatter(h4.T[0],h4.T[1],color='black',alpha=0.5)

    circle1 = plt.Circle((0, 0), 1, color='black', fill=False,alpha=0.5)    
    circle2 = plt.Circle((0, 0), 2, color='black', fill=False,alpha=0.5)    
    circle3 = plt.Circle((0, 0), 3, color='black', fill=False,alpha=0.5) 
    plt.gca().add_patch(circle1)
    plt.gca().add_patch(circle2)
    plt.gca().add_patch(circle3)
    plt.xlim(-3,3)
    plt.ylim(-3,3)
    
    plt.axis('off')

    plt.subplot(122,aspect='equal')
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    nx.draw_networkx_edges(Hbond_network,pos_circular, with_labels=False,**options)
#    for edge in list(Hbond_network.edges):
#        if (not(edge[0] in np.asarray(helix_2_ind).flatten())) | (not(edge[1] in np.asarray(helix_2_ind).flatten())):
#            nx.draw_networkx_edges(Hbond_network,pos_circular,edge_color='purple')

    for residue in resid233:
        if residue in list(Hbond_network.nodes):
            nx.draw_networkx_nodes(Hbond_network, pos_circular, node_size=20, nodelist=[residue],node_color='red')
    for residue in resid238:
        if residue in list(Hbond_network.nodes):
            nx.draw_networkx_nodes(Hbond_network, pos_circular, node_size=20, nodelist=[residue],node_color='green')
    circle1 = plt.Circle((0, 0), 1, color='black', fill=False,alpha=0.5)    
    plt.gca().add_patch(circle1)
    plt.axis('off')
    plt.suptitle(traj_note)
    plt.savefig('hbond_dat/rep/' + traj_note + '_max_network.pdf')
    plt.show()
    pathlengths = []

    for v in Hbond_network.nodes():
        spl = dict(nx.single_source_shortest_path_length(Hbond_network, v))
        for p in spl:
            pathlengths.append(spl[p])
    
    print('')
    print("average shortest path length %s" % (sum(pathlengths) / len(pathlengths)))
    print('')
#   print("length #paths")
    dist = {}
    for p in pathlengths:
        if p in dist:
            dist[p] += 1
        else:
            dist[p] = 1
    verts = dist.keys()
    #for d in sorted(verts):
    #    print('%s %d' % (d, dist[d]))
    
    print("radius: %d" % nx.radius(Hbond_network))
    print("diameter: %d" % nx.diameter(Hbond_network))
    #print("eccentricity: %s" % nx.eccentricity(Hbond_network))
    print("center: %s" % nx.center(Hbond_network))
    #print("periphery: %s" % nx.periphery(Hbond_network))
    print("density: %s" % nx.density(Hbond_network))
    print(" ")



In [6]:
def create_hbond_contact_map_per_frame(traj_note, location = '/media/scottzhuang/data/MD/', skip=10):
    top_location = traj_note + '/' + traj_note + ".ca.pdb"
    traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
    traj = md.load(location + traj_location,top = location + top_location)
    n_frames = traj.n_frames
    hbond_contact_map = pd.DataFrame(columns=['residue1','residue2','interaction'])
    hbond_network_dataframe = pd.read_csv(location + traj_note + '/hbond_analysis/hbond_network_2.csv')
    for time in range(0,n_frames):  

        hbond_network_frame_dataframe = hbond_network_dataframe[hbond_network_dataframe.traj_time == time].reset_index()
        if not(hbond_network_frame_dataframe.empty):
            for column in hbond_network_frame_dataframe.columns:
                if (column !=  'traj_time') & (column != 'residue') & (column != 'Unnamed: 0') & (column != 'index'):
                    residue1s = hbond_network_frame_dataframe.loc[(hbond_network_frame_dataframe[column] != 0)]['residue'].values
                    residue2 = int(column)
                    for residue1 in set(residue1s):

                        if hbond_contact_map[(hbond_contact_map.residue1 == residue1) & (hbond_contact_map.residue2 == residue2)].empty:
                            interaction_sum = hbond_network_frame_dataframe[(hbond_network_frame_dataframe[column] != 0) & (hbond_network_frame_dataframe.residue == residue1)].shape[0]
                            hbond_contact_map = pd.concat([hbond_contact_map,pd.DataFrame([[residue1,residue2,interaction_sum]],columns=['residue1','residue2','interaction'])],ignore_index=True)
                        else:
                            interaction_sum = hbond_network_frame_dataframe[(hbond_network_frame_dataframe[column] != 0) & (hbond_network_frame_dataframe.residue == residue1)].shape[0]
                            hbond_contact_map.loc[(hbond_contact_map.residue1 == residue1) & (hbond_contact_map.residue2 == residue2),'interaction'] += interaction_sum
        hbond_contact_map.to_csv(location + traj_note + '/hbond_analysis/hbond_contact_map_2_frame_' + str(time) + '.csv')
    print('finishing! ' + traj_note)

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(create_hbond_contact_map_per_frame)(traj_note) for traj_note in traj_notes)

def create_hbond_contact_map(traj_note, location = '/media/scottzhuang/data/MD/', skip=10):
    top_location = traj_note + '/' + traj_note + ".ca.pdb"
    traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
    traj = md.load(location + traj_location,top = location + top_location)
    n_frames = traj.n_frames
    hbond_contact_map = pd.DataFrame(columns=['residue1','residue2','interaction'])
    hbond_network_dataframe = pd.read_csv(location + traj_note + '/hbond_network.csv')
    for column in hbond_network_dataframe.columns:
        if (column !=  'traj_time') & (column != 'residue') & (column != 'Unnamed: 0'):
            residue1s = hbond_network_dataframe.loc[(hbond_network_dataframe[column] != 0)]['residue'].values
            residue2 = int(column)
            for residue1 in set(residue1s):
                if hbond_contact_map[(hbond_contact_map.residue1 == residue1) & (hbond_contact_map.residue2 == residue2)].empty:
                    interaction_sum = hbond_network_dataframe[(hbond_network_dataframe[column] != 0) & (hbond_network_dataframe.residue == residue1)].shape[0]
                    hbond_contact_map = pd.concat([hbond_contact_map,pd.DataFrame([[residue1,residue2,interaction_sum / n_frames]],columns=['residue1','residue2','interaction'])],ignore_index=True)
                else:
                    interaction_sum = hbond_network_dataframe[(hbond_network_dataframe[column] != 0) & (hbond_network_dataframe.residue == residue1)].shape[0]
                    hbond_contact_map.loc[(hbond_contact_map.residue1 == residue1) & (hbond_contact_map.residue2 == residue2),'interaction'] += interaction_sum / n_frames
    #print(hbond_contact_map)
    hbond_contact_map.to_csv(location + traj_note + '/hbond_contact_map.csv')
    print('finishing! ' + traj_note)

from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(create_hbond_contact_map)(traj_note) for traj_note in traj_notes)

# Transform to Matlab

In [22]:
traj_notess = [['5NJY_pH70_md1','5NJY_pH70_md2','5NJY_pH70_md3'],['5NJY_F238L_pH70_md1',
                 '5NJY_F238L_pH70_md2','5NJY_F238L_pH70_md3'],['5NJY_I233T_pH70_md4',
                  '5NJY_I233T_pH70_md2','5NJY_I233T_pH70_md3'],['5NJY_F238LI233T_pH70_md1',
                  '5NJY_F238LI233T_pH70_md2','5NJY_F238LI233T_pH70_md3'],['5NJY_pH46_md1','5NJY_pH46_md2','5NJY_pH46_md3'],['5NJY_F238L_pH46_md1',
                 '5NJY_F238L_pH46_md2','5NJY_F238L_pH46_md3'],['5NJY_I233T_pH46_md1',
                  '5NJY_I233T_pH46_md2','5NJY_I233T_pH46_md3'],['5NJY_F238LI233T_pH46_md1',
                  '5NJY_F238LI233T_pH46_md2','5NJY_F238LI233T_pH46_md3'],['4HFI_pH46_md1','4HFI_pH46_md2','4HFI_pH46_md3'],['4HFI_F238L_pH46_md1',
                 '4HFI_F238L_pH46_md2','4HFI_F238L_pH46_md3'],['4HFI_I233T_pH46_md1',
                  '4HFI_I233T_pH46_md2','4HFI_I233T_pH46_md3'],['4HFI_F238LI233T_pH46_md1',
                  '4HFI_F238LI233T_pH46_md2','4HFI_F238LI233T_pH46_md3'],['4NPQ_pH70_md5','4NPQ_pH70_md6','4NPQ_pH70_md7'],['4NPQ_F238L_pH70_md3',
                 '4NPQ_F238L_pH70_md4','4NPQ_F238L_pH70_md5'],['4NPQ_I233T_pH70_md3',
                  '4NPQ_I233T_pH70_md4','4NPQ_I233T_pH70_md5'],['4NPQ_F238LI233T_pH70_md3',
                  '4NPQ_F238LI233T_pH70_md4','4NPQ_F238LI233T_pH70_md5']]

In [22]:
def transform_hbond_network_data(traj_notes):
    hbond_data_trans =[]
    for traj_note in traj_notes:
        hbond_data = pd.read_csv('/media/scottzhuang/data/MD/' + traj_note + '/hbond_analysis/hbond_contact_map_2.csv')
        hbond_data_all = pd.DataFrame(columns=['residue1','residue2','hbond'])
        for ind in trans_ind:
            hbond_data_all = hbond_data_all.append(pd.DataFrame(np.concatenate(([np.zeros([590]) + ind], [trans_ind],[np.zeros([590])]), axis=0).T,columns=['residue1','residue2','hbond']),ignore_index=True)
        for residue1,residue2 in hbond_data[['residue1','residue2']].values:
            hbond_data_all.loc[(hbond_data_all.residue1 == residue1) & (hbond_data_all.residue2 == residue2),'hbond'] = hbond_data[(hbond_data.residue1 == residue1) & (hbond_data.residue2 == residue2)]['interaction'].values 
        hbond_data_all.to_csv('/media/scottzhuang/data/MD/' + traj_note + '/hbond_analysis/hbond_contact_map_complete_2.csv')
        hbond_data_trans.append(hbond_data_all['hbond'].values.reshape([590,590]))
    hbond_data_mean = np.mean(hbond_data_trans,axis=0)
    np.savetxt('/media/scottzhuang/data/MD/' + traj_notes[0] + '/hbond_analysis/' + traj_notes[0][:-4] + '_hbond_network_data_transmembrane_mean.dat',hbond_data_mean)        

In [38]:
def transform_hbond_network_data_frame(traj_notes,location = '/media/scottzhuang/data/MD/', skip=10):
    hbond_data_trans =[]
    for traj_note in traj_notes:
        top_location = traj_note + '/' + traj_note + ".ca.pdb"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top = location + top_location)
        n_frames = traj.n_frames
        for time in range(0,n_frames):  
            hbond_data = pd.read_csv('/media/scottzhuang/data/MD/' + traj_note + '/hbond_analysis/hbond_contact_map_2_frame_' + str(time) + '.csv')
            hbond_data_all = pd.DataFrame(columns=['residue1','residue2','hbond'])
            for ind in trans_ind:
                hbond_data_all = hbond_data_all.append(pd.DataFrame(np.concatenate(([np.zeros([590]) + ind], [trans_ind],[np.zeros([590])]), axis=0).T,columns=['residue1','residue2','hbond']),ignore_index=True)
            for residue1,residue2 in hbond_data[['residue1','residue2']].values:
                hbond_data_all.loc[(hbond_data_all.residue1 == residue1) & (hbond_data_all.residue2 == residue2),'hbond'] = hbond_data[(hbond_data.residue1 == residue1) & (hbond_data.residue2 == residue2)]['interaction'].values 
            hbond_data_all.to_csv('/media/scottzhuang/data/MD/' + traj_note + '/hbond_analysis/hbond_contact_map_complete_2_frame_' + str(time) + '.csv')
#            hbond_data_trans.append(hbond_data_all['hbond'].values.reshape([590,590]))
#       hbond_data_mean = np.mean(hbond_data_trans,axis=0)
#    np.savetxt('/media/scottzhuang/data/MD/' + traj_notes[0] + '/hbond_analysis/' + traj_notes[0][:-4] + '_hbond_network_data_transmembrane_mean.dat',hbond_data_mean)        

In [23]:
def transform_hbond_network_data_robust(traj_notes):
    hbond_data_trans =[]
    for (traj_note,i) in itertools.product(traj_notes,[0,1,2]):
        hbond_data = pd.read_csv('/media/scottzhuang/data/MD/' + traj_note + '/hbond_analysis/hbond_contact_map_robust_' + str(i) + '.csv')
        hbond_data_all = pd.DataFrame(columns=['residue1','residue2','hbond'])
        for ind in trans_ind:
            hbond_data_all = hbond_data_all.append(pd.DataFrame(np.concatenate(([np.zeros([590]) + ind], [trans_ind],[np.zeros([590])]), axis=0).T,columns=['residue1','residue2','hbond']),ignore_index=True)
        for residue1,residue2 in hbond_data[['residue1','residue2']].values:
            hbond_data_all.loc[(hbond_data_all.residue1 == residue1) & (hbond_data_all.residue2 == residue2),'hbond'] = hbond_data[(hbond_data.residue1 == residue1) & (hbond_data.residue2 == residue2)]['interaction'].values 
        hbond_data_all.to_csv('/media/scottzhuang/data/MD/' + traj_note + '/hbond_analysis/hbond_contact_map_complete_robust_' + str(i) + '.csv')
#        hbond_data_trans.append(hbond_data_all['hbond'].values.reshape([590,590]))
#    hbond_data_mean = np.mean(hbond_data_trans,axis=0)
#    np.savetxt('/media/scottzhuang/data/MD/' + traj_notes[0] + '/hbond_analysis/' + traj_notes[0][:-4] + '_hbond_network_data_transmembrane_mean.dat',hbond_data_mean)        

In [25]:
trans_ind = np.asarray([np.arange(193,311),np.arange(504,622),np.arange(815,933),np.arange(1126,1244),np.arange(1437,1555)]).flatten()

In [26]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(transform_hbond_network_data_robust)(traj_notes) for traj_notes in traj_notess)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [39]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(transform_hbond_network_data_frame)(traj_notes) for traj_notes in traj_notess)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]